In [2]:
cd /home/muhammadbsheikh/workspace/try

/home/muhammadbsheikh/workspace/try


In [3]:
def extract_features(model,dl):
    lbls = []
    model.eval()
    device = 'cuda:0'
    model.cuda(device)
    with torch.no_grad():
        features = None
        for batch in tqdm(dl, disable=True):
            
            images = batch[0]
            labels = batch[1]
            images = images.to(device)
            #labels = labels.to(device)

            output = model(images)
            lbls.append(labels)
            #print(labels)

            if features is not None:
                features = torch.cat((features, output), 0)

            else:
                features = output        
            

    return features.cpu().numpy(),lbls


def flatten_list(t):
    flat_list = [item for sublist in t for item in sublist]
    flat_list = np.array(flat_list)
    return flat_list

In [4]:
# cnn2fft
# check PyTorch versions
# import standard PyTorch modules
import os
os.environ['CUDA_VISIBLE_DEVICES']="1"
import torch
import torch.nn as nn
import torch.nn.functional as F
import pretrainedmodels
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter # TensorBoard support

# import torchvision module to handle image manipulation
import torchvision
from torchvision import datasets, models, transforms
import torchvision.transforms as transforms
from torch.autograd import Variable

# calculate train time, writing train data to files etc.
import time
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
from IPython.display import clear_output


from efficientnet_pytorch import EfficientNet

from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import scipy
import scipy.fft

print(torch.__version__)
print(torchvision.__version__)

#device = torch.device("cuda")
torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)     # On by default, leave it here for clarity
# torch.cuda.current_device()

# Helper class, help track loss, accuracy, epoch time, run time, 
# hyper-parameters etc. Also record to TensorBoard and write into csv, json

# import modules to build RunBuilder and RunManager helper classes
from collections  import OrderedDict
from collections import namedtuple
from itertools import product

# Read in the hyper-parameters and return a Run namedtuple containing all the 
# combinations of hyper-parameters
class RunBuilder():
  @staticmethod
  def get_runs(params):

    Run = namedtuple('Run', params.keys())

    runs = []
    for v in product(*params.values()):
      runs.append(Run(*v))
    
    return runs

class RunManager():

  def __init__(self):

    # tracking every epoch count, loss, accuracy, time
    self.epoch_count = 0
    self.epoch_loss = 0
    self.epoch_num_correct = 0
    
    self.epoch_val_loss = 0
    self.epoch_val_num_correct = 0
    self.epoch_start_time = None

    # tracking every run count, run data, hyper-params used, time
    self.run_params = None
    self.run_count = 0
    self.run_data = []
    self.run_start_time = None

    # record model, loader and TensorBoard 
    self.network = None
    self.loader = None
    self.vloader = None
    self.tb = None

  # record the count, hyper-param, model, loader of each run
  # record sample images and network graph to TensorBoard  
  def begin_run(self, run, network, loader,vloader):

    self.run_start_time = time.time()

    self.run_params = run
    self.run_count += 1

    self.network = network
    self.loader = loader
    self.vloader = vloader
    self.tb = SummaryWriter(comment=f'-{run}')

    images, labels = next(iter(self.loader))
    images, labels = images.to(device), labels.to(device)
    grid = torchvision.utils.make_grid(images)

    self.tb.add_image('images', grid)
    self.tb.add_graph(self.network, images)

  # when run ends, close TensorBoard, zero epoch count
  def end_run(self):
    self.tb.close()
    self.epoch_count = 0

  # zero epoch count, loss, accuracy, 
  def begin_epoch(self):
    self.epoch_start_time = time.time()

    self.epoch_count += 1
    self.epoch_loss = 0
    self.epoch_num_correct = 0

  # 
  def end_epoch(self):
    # calculate epoch duration and run duration(accumulate)
    epoch_duration = time.time() - self.epoch_start_time
    run_duration = time.time() - self.run_start_time

    # record epoch loss and accuracy
    loss = self.epoch_loss / len(self.loader.dataset)
    accuracy = self.epoch_num_correct / len(self.loader.dataset)
    
    vloss = self.epoch_val_loss / len(self.vloader.dataset)
    vaccuracy = self.epoch_val_num_correct / len(self.vloader.dataset)

    # Record epoch loss and accuracy to TensorBoard 
    self.tb.add_scalar('Loss', loss, self.epoch_count)
    self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
    
    # Record epoch loss and accuracy to TensorBoard 
    self.tb.add_scalar('vLoss', vloss, self.epoch_count)
    self.tb.add_scalar('vAccuracy', vaccuracy, self.epoch_count)

    # Record params to TensorBoard
    for name, param in self.network.named_parameters():
      self.tb.add_histogram(name, param, self.epoch_count)
      self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)
    
    # Write into 'results' (OrderedDict) for all run related data
    results = OrderedDict()
    results["run"] = self.run_count
    results["epoch"] = self.epoch_count
    results["loss"] = loss
    results["val_loss"] = self.epoch_val_loss
    results["val_accuracy"] = self.epoch_val_num_correct
    results["accuracy"] = accuracy
    results["epoch duration"] = epoch_duration
    results["run duration"] = run_duration

    # Record hyper-params into 'results'
    for k,v in self.run_params._asdict().items(): results[k] = v
    self.run_data.append(results)
    df = pd.DataFrame.from_dict(self.run_data, orient = 'columns')

    # display epoch information and show progress
    clear_output(wait=True)
    #display(df)

  def track_vloss_vacc(self, loss, acc):
    # multiply batch size so variety of batch sizes can be compared
    self.epoch_val_loss = loss.item()
    self.epoch_val_num_correct = acc
    
    
    # accumulate loss of batch into entire epoch loss
  def track_loss(self, loss):
    # multiply batch size so variety of batch sizes can be compared
    self.epoch_loss += loss.item() * self.loader.batch_size

  # accumulate number of corrects of batch into entire epoch num_correct
  def track_num_correct(self, preds, labels):
    self.epoch_num_correct += self._get_num_correct(preds, labels)

  @torch.no_grad()
  def _get_num_correct(self, preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()
  
  # save end results of all runs into csv, json for further a
  def save(self, fileName):

    pd.DataFrame.from_dict(
        self.run_data, 
        orient = 'columns',
    ).to_csv(f'{fileName}.csv')

    with open(f'{fileName}.json', 'w', encoding='utf-8') as f:
      json.dump(self.run_data, f, ensure_ascii=False, indent=4)

#set device
if torch.cuda.is_available():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(torch.cuda.get_device_properties(device),torch.cuda.set_device(device),torch.cuda.current_device())

#Hyperparameters

#input_size
#num_classes = 50
#learning_rate = 0.00005
BATCH_SIZE = 32
epochs = 1

# put all hyper params into a OrderedDict, easily expandable
params = OrderedDict(
    exp='6',
    lr = [0.0005],#,0.001,0.0001,0.00001], # 0.001
    batch_size = [BATCH_SIZE], # 1000
    shuffle = [True] # True,False
    # Optimizer = [Adam,NAdam,RMSProp,Adamax,SGD,Adagrad,Adadelta]
)

TRAIN_DATA_PATH = "./datax/6/train/"
TEST_DATA_PATH = "./datax/6/test/"

transform = transforms.Compose([    
    transforms.Resize(299), # preffered size for network
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
    ])

train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=transform)
test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=transform)



train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True,  num_workers=4)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=16, shuffle=True,  num_workers=4)

m = RunManager()


#prepare model
model_name = 'inceptionresnetv2' # could be fbresnet152 or inceptionresnetv2
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')

model.last_linear = nn.Identity() #freeze the model
#num_ftrs = model.last_linear.in_features
#model.last_linear = nn.Linear(num_ftrs, 50)

for p in model.parameters():
    p.requires_grad =False
    
#num_ftrs = model.last_linear.in_features

    # Here the size of each output sample is set to 2.
    # Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
#model.fc = nn.Linear(num_ftrs, 50)

PATH = "models/IRv2.pt"

torch.save(model,PATH)
#model = torch.load(PATH)

optimizer = optim.Adam(model.parameters(), lr=0.0005)

def train(model,loader,epochs=60):
    model.to(device)
    model.train()   
    print('Training...')
    for epoch in range(epochs):
        start = time.time()
        model.train()
        m.begin_epoch()
        running_loss=0

        for i,batch in enumerate(loader,0):
                images = batch[0]
                labels = batch[1]
                images = images.to(device)
                labels = labels.to(device)
                
                preds = model(images)
                loss = F.cross_entropy(preds, labels) # Adam, SGD, RSPROP

                optimizer.zero_grad()
                #loss = Variable(loss, requires_grad = True)
                #loss.backward()
                optimizer.step()

                running_loss+=loss.data

                if i%10==9:
                    end=time.time()
                    print ('[epoch %d,imgs %5d] time: %0.3f s'%(epoch+1,(i+1)*4,(end-start)))
                    #print ('[epoch %d,imgs %5d] loss: %.7f  time: %0.3f s'%(epoch+1,(i+1)*4,running_loss/100,(end-start)))
                    #tb.add_scalar('Loss', loss, epoch+1)
                    start=time.time()
                    running_loss=0    
    

train(model,train_data_loader)


print('Extracting Features...')
feat,lbls = extract_features(model,train_data_loader)
# randomforest, logisticregression, SVM , KNN, LD,  


1.8.1
0.9.1
_CudaDeviceProperties(name='GeForce GTX 1080 Ti', major=6, minor=1, total_memory=11178MB, multi_processor_count=28) None 0
Training...
[epoch 1,imgs    40] time: 1.584 s
[epoch 1,imgs    80] time: 0.963 s
[epoch 1,imgs   120] time: 0.967 s
[epoch 1,imgs   160] time: 0.966 s
[epoch 1,imgs   200] time: 0.971 s
[epoch 1,imgs   240] time: 0.967 s
[epoch 1,imgs   280] time: 0.967 s
[epoch 1,imgs   320] time: 0.965 s
[epoch 1,imgs   360] time: 0.969 s
[epoch 1,imgs   400] time: 0.971 s
[epoch 1,imgs   440] time: 0.972 s
[epoch 1,imgs   480] time: 0.972 s
[epoch 1,imgs   520] time: 0.978 s
[epoch 1,imgs   560] time: 0.970 s
[epoch 1,imgs   600] time: 0.976 s
[epoch 1,imgs   640] time: 0.971 s
[epoch 1,imgs   680] time: 0.974 s
[epoch 1,imgs   720] time: 0.973 s
[epoch 1,imgs   760] time: 0.973 s
[epoch 1,imgs   800] time: 0.973 s
[epoch 1,imgs   840] time: 0.978 s
[epoch 1,imgs   880] time: 0.974 s
[epoch 1,imgs   920] time: 0.974 s
[epoch 1,imgs   960] time: 0.977 s
[epoch 1,imgs

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/muhammadbsheikh/anaconda3/envs/pytorchpy38/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 202, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/muhammadbsheikh/anaconda3/envs/pytorchpy38/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/muhammadbsheikh/anaconda3/envs/pytorchpy38/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/muhammadbsheikh/anaconda3/envs/pytorchpy38/lib/python3.8/site-packages/torchvision/datasets/folder.py", line 178, in __getitem__
    sample = self.loader(path)
  File "/home/muhammadbsheikh/anaconda3/envs/pytorchpy38/lib/python3.8/site-packages/torchvision/datasets/folder.py"

In [28]:
a = []
for i,batch in enumerate(test_data_loader,0):
    
    lbl = batch[1]
    a.append(lbl)
    
    if i%100==99:
        break
a = flatten_list(a)
np_labels = np.array(a)
print(np.min(np_labels),np.max(np_labels),np.mean(np_labels))

0 49 24.74


In [ ]:
feat,lbls = extract_features(model,train_data_loader)

In [17]:
16*303

4848

In [35]:
#feat,lbls = extract_features(model,train_data_loader)
test_feat,test_lbls = extract_features(model,test_data_loader)
#lbls = flatten_list(lbls)
test_lbls  =flatten_list(test_lbls) # flatting the lbls

In [36]:
len(test_feat),len(lbls),len(test_lbls)

(3732, 4835, 3732)

In [ ]:
model_name = 'inceptionresnetv2' # could be fbresnet152 or inceptionresnetv2
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet').cuda()

for batch in train_data_loader:
    out = res_extractor(batch[0].cuda())
    print(out.shape)
    break


In [ ]:
c3d_model = C3D()
c3d_model.load_state_dict(torch.load('/home/muhammadbsheikh/workspace/c3d/one/c3d.pickle'))
c3d_model.cuda()
children_counter = 0
for n,c in c3d_model.named_children():
    print("Children Counter: ",children_counter," Layer Name: ",n,)
    children_counter+=1
c3d_model.modules

In [ ]:
c3ducf101_model = C3D(num_classes=101, pretrained=True)
#c3ducf101_model.load_state_dict(torch.load('/home/muhammadbsheikh/workspace/projects/C3D jfzhang95/pytorch-video-recognition/run/run_2/models/C3D-ucf101_epoch-49.pth.tar'))

checkpoint = torch.load('/home/muhammadbsheikh/workspace/projects/C3D jfzhang95/pytorch-video-recognition/run/run_2/models/C3D-ucf101_epoch-49.pth.tar')
c3ducf101_model.load_state_dict(checkpoint['state_dict'])

children_counter = 0
for n,c in c3ducf101_model.named_children():
    print("Children Counter: ",children_counter," Layer Name: ",n,)
    children_counter+=1
c3ducf101_model.modules()

In [55]:
c3ducf101_model.fc8 = nn.Identity()
c3ducf101_model

C3D(
  (conv1): Conv3d(3, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool1): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool2): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3a): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv3b): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool3): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv4a): Conv3d(256, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv4b): Conv3d(512, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv5a): Conv3d(512, 512, kernel_size=(3, 3, 3), stride=

In [56]:
inputs = torch.rand(1, 3, 16, 112, 112)
c3ducf101_model(inputs).size()
from torchsummary import summary
summary(c3ducf101_model)

torch.Size([1, 4096])

In [37]:
feat.shape,len(lbls),test_feat.shape,len(test_lbls)

((4835, 1536), 4835, (3732, 1536), 3732)

In [38]:
import pickle
import numpy as np

trainpath = '/home/muhammadbsheikh/workspace/projects/mmaction/mmaction2/train_ucf50_feature.pkl'
testpath = '/home/muhammadbsheikh/workspace/projects/mmaction/mmaction2/test_ucf50_feature.pkl'

objects = []
trainfile =  open(trainpath, "rb")
testfile =  open(testpath, "rb")
train_features = np.array(pickle.load(trainfile))
test_features = np.array(pickle.load(testfile))
        

print(train_features.shape)
print(test_features.shape)
print(feat.shape)
print(test_feat.shape)



(4835, 2048)
(1854, 2048)
(4835, 1536)
(3732, 1536)


In [39]:
#concatenation
cat_feat_train = np.concatenate((feat,train_features),axis=1)
print(cat_feat_train.shape)
cat_feat_test = np.concatenate((test_feat,test_features),axis=1)
print(cat_feat_test.shape)

(4835, 3584)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 3732 and the array at index 1 has size 1854

In [11]:
print(np.min(train_features),np.max(train_features),np.mean(train_features))
print(np.min(test_features),np.max(test_features),np.mean(test_features))
print(np.min(feat),np.max(feat),np.mean(feat))
print(np.min(test_feat),np.max(test_feat),np.mean(test_feat))

0.0 5.215954 0.36460656
0.0 1422.8102 110.30214
0.0 24.409187 0.26208088
0.0 30.845383 0.26157546


In [31]:
from sklearn import metrics
print('Train-Without FFT')
#SVM
svm = SVC(kernel='linear').fit(cat_feat_train,lbls)
svm_preds = svm.predict(cat_feat_test)
print("SVM Accuracy:",metrics.accuracy_score(test_lbls, svm_preds))

Train-Without FFT
SVM Accuracy: 0.016720604099244876


In [30]:
lbls.shape,test_lbls.shape

((4835,), (1854,))

In [10]:
from sklearn import metrics
print('Train-Without FFT')
#SVM
svm_preds = SVC(kernel='linear').fit(feat,lbls).predict(test_feat)
print("SVM Accuracy:",metrics.accuracy_score(test_lbls, svm_preds))


Train-Without FFT
SVM Accuracy: 0.46062567421790723


In [11]:
#save np network features and labels
# Use pickle for serialization or np.save for saving np objects. two options



i = 0 #video index 4797
j = 0 # neuron index 1000

def hstft(S1):
    S21, S22 = np.split(S1,2)  # 500+500
    S311, S312 = np.split(S21,2) # 500    
    S321, S322 = np.split(S22,2) # 500
    #S1.shape,S21.shape,S22.shape,S311.shape,S312.shape,S321.shape,S322.shape    
    return np.concatenate((S1,S21,S22,S311,S312,S321,S322))

def alpha(features):
    A = np.zeros((len(features),4608))
    for i,f in enumerate(features):
        h = scipy.fft.fft(f)    
        #print(f.shape,h.shape)
        #print(h.shape)
        h = hstft(h)
        #print(h.shape)
        A[i,:] = h.real
    return A

train_alpha = alpha(feat)
test_alpha = alpha(test_feat)
train_alpha.shape,test_alpha.shape

((4835, 4608), (1854, 4608))

In [12]:
X = train_alpha
Y = lbls
test_x = test_alpha
test_y = test_lbls
X.shape,Y.shape,test_x.shape,test_y.shape

((4835, 4608), (4835,), (1854, 4608), (1854,))

In [13]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

#SVM
clf = SVC(kernel='linear').fit(X,Y)
preds = clf.predict(test_x)
print("SVM Accuracy:",metrics.accuracy_score(test_y, preds))

#KNN

knn_clf = KNeighborsClassifier(n_neighbors=3).fit(X,Y)
knn_preds = knn_clf.predict(test_x)
print("KNN Accuracy:",metrics.accuracy_score(test_y, knn_preds))

#Random Forest
rf_clf = RandomForestClassifier(n_estimators=100).fit(X,Y)
rf_preds = rf_clf.predict(test_x)
print("RF Accuracy:",metrics.accuracy_score(test_y, rf_preds))


SVM Accuracy: 0.4622437971952535
KNN Accuracy: 0.3074433656957929
RF Accuracy: 0.41585760517799353
